In [ ]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, LSTM
from keras.models import Sequential
import matplotlib.pyplot as plt
import yfinance as yf

plt.style.use("fivethirtyeight")

df = yf.download('AAPL', start="2012-01-01", end='2025-02-10')


data = df[['Close']]  #  only 'Close' column

# to NumPy array
dataset = data.values
training_data_len = math.ceil(len(dataset) * 0.8)

# fit scaler only on training data
scaler = MinMaxScaler(feature_range=(0,1))
train_data = dataset[:training_data_len]  # Only training portion
scaler.fit(train_data)  # Fit only on training data

scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(dataset[training_data_len:])  


x_train, y_train = [], []
for i in range(60, len(scaled_train_data)):
    x_train.append(scaled_train_data[i-60:i, 0])
    y_train.append(scaled_train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)


model = Sequential()
model.add(LSTM(50, input_shape=(x_train.shape[1], 1), return_sequences=True))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# training
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Test Data
x_test = []
y_test = dataset[training_data_len:, :]  

for i in range(60, len(scaled_test_data)):
    x_test.append(scaled_test_data[i-60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# make predictions
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)  # reconvert to original scale

train = data[:training_data_len]
valid = data[training_data_len:]

#  matching row count
valid = valid.iloc[:len(predictions)]  # trim valid to match predictions
valid['Predictions'] = predictions.flatten()  # Convert predictions to 1D array

# checking
print(len(valid))         
print(len(predictions))  

# Display Data
print(valid.head())


In [ ]:
# Plot Results
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)

plt.plot(train['Close'], label='Train')  #  training data
plt.plot(valid['Close'], label='Validation')  #  actual prices
plt.plot(valid['Predictions'], label='Predictions')  #  predicted prices

plt.legend(loc='lower right')
plt.show()